In [1]:
path2train = "/home/e209440/data/train"
path2test = "/home/e209440/data/test"
#path2train = "/home/r120084/project/data/train"
#path2test = "/home/r120084/project/data/test"

In [2]:
%matplotlib inline
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import torch as tc
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.transforms as transforms
from torchvision import transforms, utils

from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import StepLR
from tqdm import tqdm 
import random

random.seed(7)

In [3]:
import os, glob, math,time
from scipy import misc

In [10]:
img_size = 50
sz = (img_size, img_size)


In [11]:
def process_image(img_file):
    img = misc.imread(img_file)
    img = misc.imresize(img, sz)
    img = img.transpose((2,0,1)).astype('float32') / 255.0
    return img

In [12]:
def load_traindata(path, labels):
    X_train= []
    Y_train = []
    for j in range(1):
        path = os.path.join(path2train, 'c' + str(j), '*.jpg')
        files = glob.glob(path)
        X_train.extend(process_image(im_file) for im_file in files)
        Y_train.extend([j]*len(files))
    X_train = np.array(X_train)
    X_train = tc.from_numpy(X_train).type(tc.FloatTensor)
    Y_train = np.array(Y_train)
    Y_train = tc.from_numpy(Y_train).type(tc.FloatTensor)

    return (X_train,Y_train)

In [13]:
X_train, y_train = load_traindata(path2train, labels)

In [16]:
X_train.shape,y_train.shape

(torch.Size([2489, 3, 50, 50]), torch.Size([2489]))

In [18]:
def load_testdata():
    X_test    = []
    path  = os.path.join(path2test, '*.jpg')
    files = glob.glob(path)

    idx = 0
    for im_file in tqdm(files):
        idx+=1
        if idx==20:
            break
        X_test.extend(process_image(im_file))    
        results = process_image(im_file)
        
    X_test = np.array(X_test)
    X_test = tc.from_numpy(X_test).type(tc.FloatTensor)
    return X_test


In [19]:
X_test = load_testdata()

  0%|          | 19/79726 [00:00<1:13:42, 18.02it/s]


In [20]:
X_test.shape

torch.Size([57, 50, 50])

In [ ]:
def plot_classes(labels, X, Y, img_size):
    plt.figure(figsize=(20,10))
    size_subp= len(labels)
    cnt=1
    for i in labels:
        ind = np.where(Y==str(i))[0]
        print(ind)
        image = X[ind].reshape(3,img_size,img_size).transpose((1,2,0))
        plt.subplot(5,2, cnt)
        plt.imshow(image, cmap=plt.cm.gray_r)
        plt.axis('off')
        plt.title(labels[i])
        cnt +=1

In [ ]:
plot_classes(labels, X_train, y_train, img_size)

In [21]:
few_samples = True
if few_samples:
    n_samples = 100
else : 
    n_samples = data['X_train'].shape[0]

In [22]:
X_train = X_train[:n_samples]
y_train = y_train[:n_samples]
X_test = X_test[:n_samples]


In [24]:
X_train.shape, y_train.shape, X_test.shape

(torch.Size([100, 3, 50, 50]), torch.Size([100]), torch.Size([57, 50, 50]))

In [ ]:
class CRDataset(Dataset):
    
    def __init__(self, data, target, transform):
        assert data.shape[0] == target.shape[0]
        
        self.data = data
        self.target = torch.LongTensor(target.astype(int))
        self.transform = transform
    
    def __len__(self):
        return self.target.shape[0]
    
    def __getitem__(self, i):
        data = self.transform(self.data[i, :, :, :])
        return (data, self.target[i])

